In [1]:

%pprint
import sys
if ('../py' not in sys.path): sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

from datetime import timedelta
from FRVRS import fu
from FRVRS import nu
from pandas import DataFrame
import os
import os.path as osp

In [3]:

# Get all CSVs into one data frame
if nu.pickle_exists('frvrs_logs_df'):
    frvrs_logs_df = nu.load_object('frvrs_logs_df')
    print(frvrs_logs_df.shape) # (842663, 112)
    # df = frvrs_logs_df.sample(4).dropna(axis='columns', how='all')
    # display(df.T)

(829116, 114)



# Time Spent on Task

Are all mass casualty tasks (wound packing, needle decompression, tourniquet application, triage tag determination) similar in difficulty based on time spent?

In [4]:

# Get time delta for each action type
if nu.pickle_exists('time_spent_df'):
    time_spent_df = nu.load_object('time_spent_df')
else:
    
    # Define columns for grouping
    
    
    
    # Group by patient ID
    gb = frvrs_logs_df.sort_values(['action_tick']).groupby(fu.patient_groupby_columns)
    rows_list = []
    for (session_uuid, scene_id, patient_id), patient_df in gb:
        
        # Add the logger version and scene categories
        logger_version = fu.get_logger_version(patient_df)
        is_scene_aborted = fu.get_is_scene_aborted(patient_df)
        scene_type = fu.get_scene_type(patient_df)
        
        # Get the first time the patient is noticed
        first_notice = fu.get_first_patient_interaction(patient_df)
        previous_time = first_notice
        
        # Get the actions with the patient; group by elapsed time first to get the sort order correct
        mask_series = (patient_df.action_type.isin(fu.responder_negotiations_list))
        for (action_tick, action_type), action_type_df in patient_df[mask_series].sort_values('action_tick').groupby(
            ['action_tick', 'action_type']
        ):
            if(action_type == 'INJURY_TREATED'):
                for (
                    injury_treated_id, injury_treated_required_procedure, injury_treated_severity, injury_treated_body_region
                ) in zip(
                    action_type_df.injury_treated_id, action_type_df.injury_treated_required_procedure, action_type_df.injury_treated_severity,
                    action_type_df.injury_treated_body_region
                ):
                    row_dict = {}
                    
                    # Add the groupby columns to the row dictionary
                    for cn in fu.patient_groupby_columns: row_dict[cn] = eval(cn)
                    row_dict['logger_version'] = logger_version
                    row_dict['is_scene_aborted'] = is_scene_aborted
                    row_dict['scene_type'] = scene_type
                    
                    # Add the time spent for each action
                    row_dict['first_notice'] = first_notice
                    row_dict['action_type'] = action_type
                    row_dict['injury_treated_id'] = injury_treated_id
                    row_dict['injury_treated_required_procedure'] = injury_treated_required_procedure
                    row_dict['injury_treated_severity'] = injury_treated_severity
                    row_dict['injury_treated_body_region'] = injury_treated_body_region
                    row_dict['previous_time'] = previous_time
                    row_dict['action_time'] = action_tick
                    row_dict['time_spent'] = action_tick - previous_time
                    previous_time = action_tick
                    
                    # Add the row dictionary to the list
                    rows_list.append(row_dict)
            else:
                row_dict = {}
                
                # Add the groupby columns to the row dictionary
                for cn in fu.patient_groupby_columns: row_dict[cn] = eval(cn)
                row_dict['logger_version'] = logger_version
                row_dict['is_scene_aborted'] = is_scene_aborted
                row_dict['scene_type'] = scene_type
                
                # Add the time spent for each action
                row_dict['first_notice'] = first_notice
                row_dict['action_type'] = action_type
                if(action_type == 'TAG_APPLIED'):
                    row_dict['tag_applied_type'] = action_type_df.tag_applied_type.squeeze()
                if(action_type == 'TOOL_APPLIED'):
                    row_dict['tool_applied_type'] = action_type_df.tool_applied_type.squeeze()
                row_dict['previous_time'] = previous_time
                row_dict['action_time'] = action_tick
                row_dict['time_spent'] = action_tick - previous_time
                previous_time = action_tick
                
                # Add the row dictionary to the list
                rows_list.append(row_dict)
    
    # Create a data frame from the list of row dictionaries
    time_spent_df = DataFrame(rows_list)
    nu.store_objects(time_spent_df=time_spent_df)

Pickling to /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/time_spent_df.pkl


In [14]:

# Display a sample of the data frame, dropping columns with all NaN values and transposing it
mask_series = ~time_spent_df.injury_treated_id.isnull()
df = time_spent_df[mask_series]
display(df.sample(min(4, df.shape[0])).dropna(axis='columns', how='all').T)

,4831,24540,10253,26014
session_uuid,274ce308-c366-4829-b9cc-d535b51d16d3,f02dc87e-8f04-4ddd-90fd-74dce1206bd6,6037b1d3-eff2-4317-9372-768236fbae91,ff97fc56-c803-4aa7-b419-35e0ac17d0b3
scene_id,1,2,1,1
patient_id,Mike_1 Root,Gary_3 Root,Helga_0 Root,Mike_0 Root
logger_version,1.0,1.3,1.3,1.0
is_scene_aborted,False,False,False,False
scene_type,Orientation,Triage,Triage,Triage
first_notice,45315,345545,26933,11028
action_type,INJURY_TREATED,INJURY_TREATED,INJURY_TREATED,INJURY_TREATED
injury_treated_id,L Thigh Laceration,R Chest Collapse,L Thigh Puncture,R Forearm Laceration
injury_treated_required_procedure,tourniquet,decompress,tourniquet,gauzePressure


In [15]:

def get_mean_time_spent(column_name):
    groupby_columns = [column_name]
    mask_series = (time_spent_df.scene_type == 'Triage') & (time_spent_df.is_scene_aborted == False)
    df = time_spent_df[mask_series].groupby(groupby_columns).filter(
        lambda df: not df[column_name].isnull().any()
    ).groupby(groupby_columns).time_spent.mean().sort_values().map(
        lambda x: humanize.precisedelta(timedelta(milliseconds=x))
    ).reset_index()
    df = df.rename(columns={'time_spent': 'mean_time_spent'})
    
    return df

In [16]:

# Get mean time from last action for each body region
get_mean_time_spent('injury_treated_body_region')

,injury_treated_body_region,mean_time_spent
0,neck,28.90 seconds
1,head,55.92 seconds
2,abdomen,1 minute and 0.12 seconds
3,chest,1 minute and 1.06 seconds
4,rightArm,1 minute and 10.54 seconds
5,rightLeg,1 minute and 29.95 seconds
6,leftLeg,1 minute and 32.29 seconds
7,leftArm,1 minute and 52.47 seconds


In [17]:

# Get mean time from last action for each treatment severity
get_mean_time_spent('injury_treated_severity')

,injury_treated_severity,mean_time_spent
0,high,1 minute and 8.78 seconds
1,low,1 minute and 19.00 seconds
2,medium,1 minute and 26.65 seconds


In [18]:

# Get mean time from last action for each treatment procedure
get_mean_time_spent('injury_treated_required_procedure')

,injury_treated_required_procedure,mean_time_spent
0,none,10.66 seconds
1,airway,41.05 seconds
2,decompress,42.07 seconds
3,woundpack,1 minute and 11.25 seconds
4,gauzePressure,1 minute and 18.55 seconds
5,tourniquet,1 minute and 41.99 seconds


In [21]:

def show_statistics(column_name):
    mask_series = (time_spent_df.scene_type == 'Triage') & (time_spent_df.is_scene_aborted == False) & ~time_spent_df[column_name].isnull()
    df = nu.get_statistics(time_spent_df[mask_series], ['time_spent'])
    # .mean().sort_values().map(
    #     lambda x: humanize.precisedelta(timedelta(milliseconds=x))
    # ).reset_index()
    # df = df.rename(columns={'time_spent': 'mean_time_spent'})
    
    return df

In [22]:

# Get mean time from last action for each treatment procedure
show_statistics('injury_treated_required_procedure')

,scene_id,logger_version,first_notice,previous_time,action_time,time_spent
mean,1.736407,1.135907,2.632933e+05,4.009003e+05,4.815679e+05,80667.571118
mode,NaN,NaN,NaN,NaN,NaN,2701.000000
median,NaN,NaN,NaN,NaN,NaN,12699.000000
SD,1.543970,0.149353,6.210286e+05,6.422739e+05,6.420646e+05,116133.516122
min,1.000000,1.000000,7.062000e+03,7.062000e+03,2.755000e+04,0.000000
25%,1.000000,1.000000,2.204700e+04,9.967675e+04,1.976075e+05,5714.250000
50%,1.000000,1.000000,1.008290e+05,2.856130e+05,3.724470e+05,12699.000000
75%,2.000000,1.300000,3.183140e+05,4.929990e+05,5.702258e+05,139451.500000
max,16.000000,1.300000,6.931919e+06,7.144298e+06,7.152352e+06,725845.000000


In [123]:

# Get mean time from last action for each injury type, quickest
get_mean_time_spent('injury_treated_id').head(5)

,injury_treated_id,mean_time_spent
0,L Chest Collapse,7.31 seconds
1,R Bicep Puncture,15.07 seconds
2,L Bicep Puncture,18.41 seconds
3,L Stomach Puncture,24.58 seconds
4,R Side Puncture,28.67 seconds


In [122]:

# Get mean time from last action for each injury type, slowest
get_mean_time_spent('injury_treated_id').tail(5)

,injury_treated_id,mean_time_spent
19,R Shin Amputation,1 minute and 53.55 seconds
20,L Forearm Laceration,1 minute and 54.26 seconds
21,Forehead Scrape,2 minutes and 3.53 seconds
22,R Shoulder Puncture,2 minutes and 13.40 seconds
23,R Wrist Amputation,2 minutes and 35.27 seconds


In [67]:

# Get mean time from last action for each action type
get_mean_time_spent('action_type')

,action_type,time_spent
0,TOOL_APPLIED,12.28 seconds
1,PULSE_TAKEN,48.06 seconds
2,TAG_APPLIED,48.24 seconds
3,INJURY_TREATED,1 minute and 20.55 seconds


In [68]:

# Get mean time from last action for each tool type
get_mean_time_spent('tool_applied_type')

,tool_applied_type,time_spent
0,Gauze_Pack,6.49 seconds
1,Tourniquet,10.26 seconds
2,Gauze_Dressing,20.22 seconds
3,Naso,30.91 seconds
4,Needle,51.14 seconds


In [69]:

# Get mean time from last action for each tag type
get_mean_time_spent('tag_applied_type')

,tag_applied_type,time_spent
0,red,19.95 seconds
1,yellow,35.71 seconds
2,black,37.00 seconds
3,gray,46.63 seconds
4,green,1 minute and 56.26 seconds


In [124]:

def get_patient_variability(columns_dict, column_name):
    variability_df = DataFrame(time_spent_df.patient_id.unique().tolist(), columns=['patient_id'])
    for column_value, time_spent_rename in columns_dict.items():
        mask_series = (time_spent_df[column_name] == column_value)
        groupby_columns = ['patient_id']
        df = time_spent_df[mask_series].groupby(groupby_columns).filter(
            lambda df: not df.time_spent.isnull().any()
        ).groupby(groupby_columns).time_spent.std().sort_values(ascending=False).reset_index()
        mask_series = df.time_spent.isnull()
        df = df[~mask_series].rename(columns={'time_spent': time_spent_rename})
        df[time_spent_rename] = df[time_spent_rename].map(
            lambda x: '±' + humanize.precisedelta(timedelta(milliseconds=x))
        )
        variability_df = variability_df.merge(df, on='patient_id', how='outer')
    
    return variability_df

In [125]:

# Get patients with the greatest variability in actions
columns_dict = {
    'TOOL_APPLIED': 'variability_in_applying_tools',
    'PULSE_TAKEN': 'variability_in_taking_pulses',
    'TAG_APPLIED': 'variability_in_applying_tags',
    'INJURY_TREATED': 'variability_in_treating_injuries',
}
get_patient_variability(columns_dict, 'action_type').head(5)

,patient_id,variability_in_applying_tools,variability_in_taking_pulses,variability_in_applying_tags,variability_in_treating_injuries
0,Gary_0 Root,±10.49 seconds,±14.04 seconds,±12.53 seconds,±27.85 seconds
1,Bob_0 Root,±1 minute and 33.48 seconds,±1 minute and 43.82 seconds,±1 minute and 30.22 seconds,NaN
2,Bob_9 Root,NaN,±3 minutes and 16.06 seconds,±4 minutes and 4.01 seconds,NaN
3,Gary_1 Root,±1 minute and 18.74 seconds,±1 minute and 21.21 seconds,±1 minute and 7.81 seconds,±58.91 seconds
4,Gary_3 Root,±2 minutes and 10.45 seconds,±1 minute and 9.49 seconds,±40.53 seconds,±1 minute and 11.23 seconds


In [126]:

# Get patients with the greatest variability in tags
columns_dict = {
    'red': 'variability_in_applying_red',
    'yellow': 'variability_in_applying_yellow',
    'black': 'variability_in_applying_black',
    'gray': 'variability_in_applying_gray',
    'green': 'variability_in_applying_green',
}
get_patient_variability(columns_dict, 'tag_applied_type').head(5)

,patient_id,variability_in_applying_red,variability_in_applying_yellow,variability_in_applying_black,variability_in_applying_gray,variability_in_applying_green
0,Gary_0 Root,±15.08 seconds,±11.84 seconds,±12.72 seconds,±17.84 seconds,±3.68 seconds
1,Bob_0 Root,±2 minutes and 22.97 seconds,NaN,±1 minute and 29.50 seconds,±1 minute and 8.15 seconds,±1 minute and 53.27 seconds
2,Bob_9 Root,NaN,±3 minutes and 27.49 seconds,NaN,±4 minutes and 19.91 seconds,±4 minutes and 13.21 seconds
3,Gary_1 Root,±1 minute and 46.77 seconds,±6.18 seconds,±41.77 seconds,±1 minute and 7.96 seconds,±2.78 seconds
4,Gary_3 Root,±22.53 seconds,±36.77 seconds,±2 minutes and 17.37 seconds,±1 minute and 9.98 seconds,NaN


In [128]:

# Get patients with the greatest variability in treatment
columns_dict = {
    'none': 'variability_in_applying_none',
    'airway': 'variability_in_applying_airway',
    'decompress': 'variability_in_applying_decompress',
    'woundpack': 'variability_in_applying_woundpack',
    'gauzePressure': 'variability_in_applying_gauzePressure',
    'tourniquet': 'variability_in_applying_tourniquet',
}
df = get_patient_variability(columns_dict, 'injury_treated_required_procedure')
mask_series = ~df.variability_in_applying_woundpack.isnull()
df[mask_series].head(4).T

,9,10,14,20
patient_id,Lily_4 Root,Mike_5 Root,Mike_2 Root,Helga_0 Root
variability_in_applying_none,NaN,NaN,NaN,NaN
variability_in_applying_airway,NaN,NaN,NaN,NaN
variability_in_applying_decompress,NaN,NaN,NaN,NaN
variability_in_applying_woundpack,±2 minutes and 1.97 seconds,±1 minute and 12.65 seconds,±1 minute and 58.33 seconds,±54.67 seconds
variability_in_applying_gauzePressure,NaN,±2 minutes and 19.81 seconds,NaN,NaN
variability_in_applying_tourniquet,NaN,±2 minutes and 7.87 seconds,NaN,±2 minutes and 1.18 seconds


In [ ]:

# Get patients with the greatest variability in tags or treatment; time spent on patient; time spent on tool decision or tag decision; return visits to patient and tag changes